In [25]:
import string
from collections import Counter
import time

### So we start with a word list for most common english words. We can start with an inbuilt dictionary or some different word list as well.

In [26]:
#List of most common 5 letter words
word_list = open("words.txt").read().split()

### First we assign the limit to the length of the word and allowed attempts. The ***counts*** variable finds the number of occurences of a character in the entire word_list

In [42]:
word_length = 5 # length of words
allowed_attempts = 6 # number of allowed attempts
counts = Counter(letter for word in word_list for letter in word)
counts

Counter({'w': 505,
         'h': 814,
         'i': 1592,
         'c': 964,
         't': 1585,
         'e': 3009,
         'r': 1910,
         'a': 2348,
         'b': 715,
         'o': 1915,
         'u': 1089,
         'l': 1586,
         'd': 1181,
         's': 3033,
         'f': 561,
         'g': 679,
         'n': 1285,
         'k': 596,
         'y': 886,
         'p': 955,
         'v': 318,
         'm': 843,
         'q': 53,
         'j': 89,
         'x': 139,
         'z': 135})

### get_weight() gives weight to every word. Since in English alphabet some characters occur more frequently than others, some words are more common than others. Like "sugar" is more common according to this rule than say "yawn".

### We only need to add contributions from unique characters so that "asses" has less weight than more common words like "arose".  Also since we tend to understand things better if it is between 0 and 1, we simply make a division by total number to get a value.

### Remember, this is not the best way to get most common words but it works fine. 

In [28]:
def get_weight(word_):
    """
    word_ : Takes the input word and gives it weight.
    Higher the number, more weight(not necessarily more common) it has.
    """
    weight = 0
    unique_word = "".join(set(word_))
    for char in unique_word:
        weight += counts[char]
        total = sum(counts.values())
    return weight/total

In [47]:
print(get_weight("sugar"))
print(get_weight("yawn"))
print(get_weight("asses"))
print(get_weight("arose"))

0.31471252388396737
0.17453534827166928
0.2914712523883967
0.4243529616119507


### sorted_word_list() simply sorts the list of words based on it weight in ascending or desending order(depending on reverse = True/False.

In [29]:
def sorted_word_list(list_):
    """
    list_ : Takes the input word list
    Sorts the words in the list_ in descending order(lowest to highest)
    """
    sorted_list=[]
    for item in list_:
        weight = get_weight(item)
        tmp_list = [item,weight]
        sorted_list.append(tmp_list)
    return sorted(sorted_list, key=lambda x:x[1],reverse=True)

In [48]:
tmp_list = ["ultra", "sugar","asses","arose"]
sorted_word_list(tmp_list)

[['arose', 0.4243529616119507],
 ['sugar', 0.31471252388396737],
 ['ultra', 0.29591801285391695],
 ['asses', 0.2914712523883967]]

### display_common_words() displays the top words based on their weight.

In [30]:
def display_common_words(list_,n_words = 15):
    """
    Displays most common words after each guesses.
    """
    
    sorted_list = sorted_word_list(list_)
    first_n_list = sorted_list[:n_words] #displays top 15 entries for the word
    for i in range(len(first_n_list)):
        word = first_n_list[i][0]
        weight = first_n_list[i][1]
        print(f"{word} : {weight:.3f}")

In [49]:
display_common_words(tmp_list, 3)

arose : 0.424
sugar : 0.315
ultra : 0.296


In [31]:
#Take input for first word
def input_word():
    """
    Takes the input from the user
    """
    while True:
        word = input("Enter Your Guess: ")
        if len(word) == word_length and word.lower() in word_list:
            break
    return word.lower()

In [50]:
input_word()

Enter Your Guess: arose


'arose'

### Now we need to take response from the output of wordle to make the new guess. So once the word is entered, wordle gives three color-coded response. Green means that the character entered is present in the word and is at correct position, Yellow means while the character is present, it is in wrong position and finally black means the character is not in the word. 

### So we take the input from user G: Green, Y: Yellow and ?: Grey.

In [51]:
def input_response():
    """
    Collects the response from the wordle.
    
    G: Green(Correct character at correct position)
    Y: Yellow(Correct character at wrong position)
    ?: Black (wrong character)
    
    """
    
    print(f"Type the color-coded response from Wordle for the last word (G,Y,?): ")
    while True:
        response = input("Response from Wordle: ")
        if len(response) == word_length and set(response) <= {"G", "Y", "?"}:
            break
        else:
            print(f"Invalid response: {response}")
    return response

In [53]:
input_response()

Type the color-coded response from Wordle for the last word (G,Y,?): 
Response from Wordle: G?Y?G


'G?Y?G'

### So now we have everything set up. Now we need to find a way to make a pair-wise search of each character to all the alphabets and remove/add the alphabets depending on the response. Say, we found that the first position has green for letter "a" then we do not need to look for words which has anything other than "a" in the first place. Similarly we do not care about letters which do not exist. For yellow, we simply remove the word to occured at that position. 

### We need to match the characters of the words to some sort of list of all alphabets and change the list accordingly. 

 `
alphabets_list = [set(string.ascii_lowercase) for _ in range(word_length)] 
 `
### creates a list of set of alphabets. i.e we have a set of alphabets from a to z in a list. We have 5 of them, one for each position. 

In [33]:
def match_word(word, word_):
    """
    word: The word to be matched
    word_: The set of alphabets 
    
    This matches the characters of the word to the set of alphabets.
    """
    for word_char, alphabet in zip(word, word_):
        if word_char not in alphabet:
            return False
    return True

### We need to filter the new list of words after response from wordle. match() returns word from the list of possible words after each guess which passes the test of being in the alphabet list. The alphabet list is updated in each turn so list of words matching also changes

In [34]:
def match(alphabets_list, possible_words):
    """
    This function filters the new words from the word_list
    after response from wordle is given.
    """
    return [word for word in possible_words if match_word(word, alphabets_list)]

### Finally we collect all of the above to do things iteratively.  we start with an exact copy of word_list which currently has all the possible words. we let things run for 6 times as you only get 6 attempts. Along the way we display top 15 words and check for response and finally update our possible words list for next iteration. 

In [35]:
def solve_wordle():
    
    """
    Solves the wordle interactively
    """
    
    possible_words = word_list.copy()
    alphabets_list = [set(string.ascii_lowercase) for _ in range(word_length)]
    for attempt in range(1, allowed_attempts + 1):
        print(f"Attempt {attempt} with {len(possible_words)} possible words: ")
        display_common_words(possible_words) #displays the words
        word = input_word() #takes input from user
        response = input_response() #takes response
        for index, letter in enumerate(response):
            if letter == "G": #Fix the position to have that character
                alphabets_list[index] = {word[index]}
            elif letter == "Y":#Remove the char from that index(position)
                try:
                    alphabets_list[index].remove(word[index])
                except KeyError:
                        pass
            elif letter == "?": #Remove the char
                for vector in alphabets_list:
                    try:
                        vector.remove(word[index])
                    except KeyError:
                        pass
        possible_words = match(alphabets_list, possible_words)

In [ ]:
## uncomment and run the following line.

#solve_wordle()